<a href="https://colab.research.google.com/github/Kandacekay/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,244 kB]
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main amd64 Packages [1,152 kB]
Get:13 http://security.u

In [ ]:
# import os
# import findspark

# # Specify the Spark installation path
# spark_path = "C:/Spark/Spark3"

# # Specify the Hadoop installation path
# hadoop_path = "C:/Hadoop"

# # Set the environment variables
# os.environ["SPARK_HOME"] = spark_path
# os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk1.8.0_281"  # Adjust the Java path as per your installation
# os.environ["HADOOP_HOME"] = hadoop_path
# os.environ["PATH"] = os.path.join(hadoop_path, "bin") + ";" + os.environ["PATH"]

# # Add the Spark bin and PySpark to the system path
# os.environ["PATH"] = os.path.join(spark_path, "bin") + ";" + os.environ["PATH"]
# os.environ["PATH"] = os.path.join(spark_path, "python") + ";" + os.environ["PATH"]

# # Initialize findspark
# findspark.init()

# # Import SparkSession
# from pyspark.sql import SparkSession

# # Create a SparkSession
# spark = SparkSession.builder.master("local").appName("YourAppName").getOrCreate()


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv("file:///" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")



In [5]:
df.columns

['id',
 'date',
 'date_built',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view']

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query_3 = """
SELECT year(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year(date)
ORDER BY year
"""
result_3 = spark.sql(query_3)
result_3.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query_4 = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY year_built
"""
result_4 = spark.sql(query_4)
result_4.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [8]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query_5 = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY year_built
"""
result_5 = spark.sql(query_5)
result_5.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query_6 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
"""
result_6 = spark.sql(query_6)
result_6.show()

end_time = time.time()
run_time = round(end_time - start_time, 2)
print("Query runtime:", run_time, "seconds")

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

Query runtime: 0.69 seconds


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Cache the temporary table home_sales
spark.sql("CACHE TABLE home_sales")

# Cached Query
start_time_cached = time.time()

query_cached = """
SELECT DISTINCT view
FROM home_sales
WHERE price >= 350000
"""
result_cached = spark.sql(query_cached)
result_cached.show()

end_time_cached = time.time()
run_time_cached = round(end_time_cached - start_time_cached, 2)
print("Cached Query runtime:", run_time_cached, "seconds")

+----+
|view|
+----+
|  31|
|  85|
|  65|
|  53|
|  78|
|  34|
|  81|
|  28|
|  76|
|  26|
|  27|
|  44|
|  12|
|  91|
|  22|
|  93|
|  47|
|   1|
|  52|
|  13|
+----+
only showing top 20 rows

Cached Query runtime: 0.56 seconds


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet('date_built', mode='overwrite')

In [16]:
# 11. Read the parquet formatted data.
df_read = spark.read.parquet('date_built')
df_read.show(3)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
only showing top 3 rows



In [17]:
# 12. Create a temporary table for the parquet data.
df_read.createOrReplaceTempView("parquet_temp_table")


In [20]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query_result = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM parquet_temp_table
    WHERE view IS NOT NULL
    GROUP BY view
    HAVING AVG(price) >= 350000
""")

runtime = time.time() - start_time

# Display the query result
query_result.show()

# Print the runtime
print("--- %s seconds ---" % runtime)

+----+----------+
|view| avg_price|
+----+----------+
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  81|1053472.79|
|  76|1058802.78|
|  91|1137372.73|
|  93|1026006.06|
|  52| 733780.26|
|  86|1070444.25|
|  94| 1033536.2|
|  57|  734340.5|
|  54| 798684.82|
|  96|1017815.92|
|  92| 970402.55|
|  64| 767036.67|
|  61| 746877.59|
|  88|1031719.35|
|  72| 780914.67|
|  59|  791453.0|
+----+----------+
only showing top 20 rows

--- 0.038668155670166016 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [22]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
    print("home_sales is cached.")
else:
    print("home_sales is not cached.")

home_sales is not cached.
